In [1]:
import os, sys
import pandas as pd
from pydub import AudioSegment

from preprocessing_code.Preprocessing_Cutting_Merging import (
    get_wav_time, get_second_part_wav, 
)

def wav_combine(path_wav_merge_list, path_wav_out):
    sounds = [AudioSegment.from_wav(x) for x in path_wav_merge_list]
    playlist = AudioSegment.empty()
    for sound in sounds: playlist += sound
    playlist.export(path_wav_out, format="wav")


## segment form csv or excel 

In [2]:
# The cut audio directory, which is also the audio directory to be merged.
path_segment = os.path.join(os.getcwd(), 'preprocessing_code', 'data_segment')
path_dir_excel = os.path.join(os.getcwd(), 'preprocessing_code', 'data_time')
path_dir_audio = os.path.join(os.getcwd(), '..', 'daic_woz', 'audio')
if not os.path.exists(path_segment): os.makedirs(path_segment)

a_l = []  # Record the number of audio cuts
print('Start cutting the audio!')

for root, dir, files in os.walk(path_dir_excel):
    files = sorted(files)
    for file in files:
        if file.endswith('.xlsx') and file[0].isdigit() : # * check daic_woz excel 
            user_id_str = file.split('_TRANSCRIPT')[0]
            audio_file = f'{user_id_str}_AUDIO.wav'
            path_audio = os.path.join(path_dir_audio, audio_file)
            path_excel = os.path.join(path_dir_excel, file)
            time_all = get_wav_time(path_audio)

            print(f"The audio  is {audio_file}, the file is {file}")
            df = pd.read_excel(path_excel, usecols=['start_time', 'stop_time'])

            index = 1  # Cut serial number names, starting with serial number 1
            k = 0
            # k_l = 0  # Total number of audio cuts recorded
            for j in df:
                l = len(df)
                while k <= l - 1:
                    dict_data = df.iloc[k].to_dict()
                    start_time = float(dict_data['start_time'])
                    stop_time = float(dict_data['stop_time'])

                    audio_segment = os.path.join(path_segment, f"{audio_file[:-len('.wav')]}_{index}.wav" )  
                    get_second_part_wav(path_audio, start_time, stop_time, audio_segment)

                    index += 1
                    k += 1
        a_l.append(k)
        # print(a_l)
        print("Cutting out %d audio" % l)

print('A total of %d audio cuts' % sum(a_l))
print(a_l)
print('Audio cutting complete!')


Start cutting the audio!


NameError: name 'k' is not defined

## sorted file_path 

In [3]:
path_wav_list = [x for x in os.listdir(path_segment) if x.endswith('.wav') and x[0].isdigit()]
# * example file wav name : 300_AUDIO_14.wav 
path_wav_list = sorted(path_wav_list, key=lambda x: 
                    (int(x.split('_')[0]), int(x.split('_')[-1].split('.')[0])))
path_wav_list = [os.path.join(path_segment, x) for x in path_wav_list]
# path_wav_list

In [4]:
# Initial value setting

print('Start merging audio!')
num_merge = 5 
err_id_list = [341, 393, 397, 459]

path_dir_audio_combine = os.path.join(os.getcwd(), 'preprocessing_code','data_audio_combine')
if not os.path.exists(path_dir_audio_combine): os.makedirs(path_dir_audio_combine)

filename_list = [os.path.basename(path) for path in path_wav_list]
user_id_list = [os.path.basename(path) for path in filename_list]
user_id_list = sorted(set([int(x.split('_')[0]) for x in user_id_list]))

# user_id_list
for user_id in user_id_list:
    if not user_id in err_id_list:
        filename_user_list = [x for x in filename_list if x.startswith(str(user_id))]
        filename_user_list = sorted(filename_user_list, 
                                    key=lambda x: int(x.split('_')[-1].split('.')[0]))
        path_sel_list = [os.path.join(path_segment, x) for x in filename_user_list]
        for idx, file_merge_list in enumerate([path_sel_list[i:i + num_merge] for i in range(0, len(path_sel_list), num_merge)]):
            out_path = os.path.join(path_dir_audio_combine, f"{user_id}_{idx+1}.wav")
            wav_combine(path_wav_merge_list = file_merge_list, 
                        path_wav_out = out_path)
print('Audio merge complete!')


Start merging audio!
Audio merge complete!


## merging audio

In [ ]:
print('*' * 50)
print('Start merging audio!')

err_id_list = [341, 393, 397, 459]

path_dir_audio_combine = os.path.join(os.getcwd(), 'preprocessing_code','data_audio_combine')
if not os.path.exists(path_dir_audio_combine): os.makedirs(path_dir_audio_combine)

# Initial value setting
n = 5 



## understand function preprocessing 

In [5]:

path_wav_test = os.path.join(os.getcwd(), '..', 'daic_woz', 'audio', '300_AUDIO.wav')
# sorted(os.listdir(path_wav_test))
print(f'path_wav_test : {path_wav_test}')
print(f'get_wav_time(path_wav_test) : {get_wav_time(path_wav_test)}')


path_wav_test : /Users/pataweeprakrankamanant/Desktop/Depression_Recognition/../daic_woz/audio/300_AUDIO.wav
get_wav_time(path_wav_test) : 648.5


In [13]:
path_excel_test = '/Users/pataweeprakrankamanant/Desktop/Depression_Recognition/preprocessing_code/data_time/300_TRANSCRIPT.xlsx'

df = pd.read_excel(path_excel_test, usecols=['start_time', 'stop_time'])

In [14]:
df

,start_time,stop_time
0,62.328,63.178
1,68.978,70.288
2,75.028,78.128
3,83.808,84.588
4,88.458,89.968
5,90.278,93.568
6,94.738,95.298
7,96.588,97.278
8,102.428,103.268
9,104.278,105.558
